Import Libraries

In [29]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.data
import networkx as nx

Download Cora Dataset

In [30]:
dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


Get the DGL graph

In [31]:
g = dataset[0]

Convert to a networkx graph

In [32]:
G = dgl.to_networkx(g)
# nx.write_gexf(G,"dgl_cora.gexf")

Get the ids of the largest connected component

In [33]:
components = nx.strongly_connected_components(G)
largest_component = max(components, key=len)

Get the ids of the disconnected nodes

In [34]:
disconnected_nodes = torch.LongTensor(list(set(G.nodes()).difference(largest_component)))

remove disconnected nodes from DGL graph

In [35]:
g.remove_nodes(disconnected_nodes)
g.num_nodes()

2485

Save the largest connected component

In [54]:
G_c = dgl.to_networkx(g)
nx.write_gexf(G_c,"dgl_cora_connected.gexf")

In [36]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 0, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
Edge features
{}


METIS Partitioning

In [99]:
parts = dgl.metis_partition_assignment(g,3)

Convert a graph into a bidirected graph: 0.000 seconds, peak memory: 8.695 GB
Construct multi-constraint weights: 0.000 seconds, peak memory: 8.695 GB
Metis partitioning: 0.001 seconds, peak memory: 8.695 GB


[00:05:11] /opt/dgl/src/graph/transform/metis_partition_hetero.cc:87: Partition a graph with 2485 nodes and 10138 edges into 3 parts and get 416 edge cuts


In [106]:
part_dict = {}
for id,part_id in enumerate(parts):
    part_dict[id] = part_id
# for part in range(len(parts)):
#     orig_ids = parts[part].ndata['orig_id']
#     for id in orig_ids:
#         part_dict[id.item()] = part


In [107]:
nx.set_node_attributes(G_c,part_dict,name="partition")
nx.write_gexf(G_c,"dgl_cora_connected_parts.gexf")

In [91]:
nx.write_gexf(dgl.to_networkx(parts[7]),"part7.gexf")

In [103]:
import metis

In [104]:
_, parts = metis.part_graph(G_c,3)

In [41]:
print(sum(g.ndata['test_mask']==True))
print(sum(g.ndata['val_mask']==True))
print(sum(g.ndata['train_mask']==True))

tensor(915)
tensor(459)
tensor(122)


In [42]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [53]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.146 (best 0.146), test acc: 0.162 (best 0.162)
In epoch 5, loss: 1.901, val acc: 0.377 (best 0.486), test acc: 0.375 (best 0.486)
In epoch 10, loss: 1.837, val acc: 0.270 (best 0.486), test acc: 0.274 (best 0.486)
In epoch 15, loss: 1.755, val acc: 0.327 (best 0.486), test acc: 0.330 (best 0.486)
In epoch 20, loss: 1.648, val acc: 0.412 (best 0.486), test acc: 0.445 (best 0.486)
In epoch 25, loss: 1.525, val acc: 0.540 (best 0.540), test acc: 0.534 (best 0.534)
In epoch 30, loss: 1.384, val acc: 0.619 (best 0.619), test acc: 0.630 (best 0.630)
In epoch 35, loss: 1.232, val acc: 0.658 (best 0.658), test acc: 0.664 (best 0.659)
In epoch 40, loss: 1.073, val acc: 0.684 (best 0.684), test acc: 0.683 (best 0.683)
In epoch 45, loss: 0.914, val acc: 0.710 (best 0.710), test acc: 0.710 (best 0.710)
In epoch 50, loss: 0.761, val acc: 0.728 (best 0.730), test acc: 0.726 (best 0.725)
In epoch 55, loss: 0.623, val acc: 0.741 (best 0.741), test acc: 0.736 (best 0